In [ ]:
#!pip install yfinance==0.2.54

## Introduction - Allocation of shares to the selected assets

For $𝒏$ stocks with average monthly return $𝝁_𝒊$ per dollar spent on stock $𝒊$ and covariance $𝝈_𝒊𝒋$  between stocks $𝒊$ and $𝒋$, the portfolio optimization problem, given a budget of $𝑩$ dollars, involves determining the optimal number of shares $𝒙_𝒊$ of each stock $𝒊$ purchased at price $𝒑_𝒊$ per share. This can be formulated as:

$$
\min_{}  ( q\sum_{i=1}^{n} \sum_{j=1}^{n} \sigma_{ij} (p_i x_i)(p_j x_j) - \sum_{i=1}^{n} \mu_i (p_i x_i))
$$

**Subject to:**
$$
0.997*B \leq \sum_{i=1}^{n} (p_i x_i) \leq B  \quad \text{(Budget Constraint)}
$$

$$
x_i \in \{0, 1, 2, 3, 4, \dots\}, \quad \forall i \in \{1, 2, 3, 4, \dots, n\}  \quad \text{(Non-Negativity Constraint)}
$$

- $q > 0$ controls the risk appetite of the decision maker


## Problem reformulation - To encode integer variables into binary

We can replace certain terms from the original formulation,  

$$
p_i x_i = w_i B
$$

$$
p_j x_j = w_j B
$$

and the Budget Constraint as:  

$$
\sum_{i=1}^{n} w_i = 1 \quad (\text{Utilizing total budget})
$$

where, $B$ is the budget, and $w_i$ in simple terms is the percentage of the Total Budget.

For example, let's say $w_i = 0.07$, then this represents that you invest $7\%$ of your total budget on stock $i$  

$$
w_i = w_{i, \min} + \Delta w_i \cdot w_{i}^{'}
$$

where  

$$
\Delta w_i = (w_{i,\max} - w_{i,\min})
$$

and  

$$
w_{i}^{'} = \sum_{k=1}^{K} 2^{k-1} x_{i, k}  P_K
$$

$K$ is the number of binary variables that we want to assign to a stock to represent the percentage of budget,  

$$
P_K = \frac{1}{2^K}
$$

and $w_{i,\min}$, $w_{i,\max}$ allow us to set the maximum and minimum investment limits for each stock.






## New Formulation

For **n** stocks with average monthly return **$\mu_i$** per dollar spent on stock **$i$** and covariance **$\sigma_{ij}$** between stocks **$i$** and **$j$**, the portfolio optimization problem, given a budget of **$B$** dollars, involves determining the investment percentage of total budget **$w_i$** of each stock **$i$**  

$$
\min \left( q \sum_{i=1}^{n} \sum_{j=1}^{n} \sigma_{ij} (w_i B ) (w_j B ) - \sum_{i=1}^{n} \mu_i (w_i B) \right)
$$  

subject to:  
$$
\sum_{i=1}^n w_i = 1
$$


In [1]:
B = 1099888 # set the budget here

q = 0.5 # set the risk apetite

## Sample data 

In [2]:
import yfinance as yf
## passing start and end date
start_date = "2023-01-01"
end_date = "2025-01-01"

assets = ["JPM", "GS", "MSFT", "AAPL", "KO", "WMT", "MCD", "NKE"] ## list of assets

## code to give a call to yfinance api request.
stocks_data = yf.download(assets, start = start_date, end = end_date, multi_level_index = False)['Close']
stocks_data ## df of Adj. closing price

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  8 of 8 completed


Ticker,AAPL,GS,JPM,KO,MCD,MSFT,NKE,WMT
Date,,,,,,,,
2023-01-03,123.632530,327.647095,127.490311,59.216164,252.362442,235.711716,115.257538,46.589115
2023-01-04,124.907707,329.047760,128.679184,59.187939,252.419800,225.400925,117.645195,46.641018
2023-01-05,123.583099,325.319092,128.650665,58.510643,250.290726,218.720535,117.072556,46.482048
2023-01-06,128.130203,329.407349,131.112457,59.639465,257.269745,221.298233,120.867531,47.620815
2023-01-09,128.654160,334.063416,130.570679,58.896324,255.150253,223.452866,121.178146,47.027103
...,...,...,...,...,...,...,...,...
2024-12-24,257.916443,582.789978,241.064987,62.840000,293.989990,439.329987,76.790001,92.680000
2024-12-26,258.735504,581.229980,241.890717,62.570000,294.799988,438.109985,76.940002,92.790001
2024-12-27,255.309296,576.179993,239.930847,62.450001,293.619995,430.529999,76.419998,91.660004


In [3]:
price = stocks_data.iloc[-1].tolist()
price

[250.1449737548828,
 572.6199951171875,
 238.4783477783203,
 62.2599983215332,
 289.8900146484375,
 421.5,
 75.66999816894531,
 90.3499984741211]

In [4]:
import numpy as np
stock_return = stocks_data.pct_change() ## applying percentage change on the adj closing price

mu = stock_return.mean() ## mean of the df
sigma = stock_return.cov() ## covariance of the df
sigma = np.array(sigma)
mu = np.array(mu)
n = len(mu)  #num_assets

In [5]:
mu

array([ 0.00149762,  0.00123719,  0.00134746,  0.00013419,  0.00032763,
        0.00126254, -0.00065842,  0.00137833])

## Start coding below

## Trying for New formulation

In [6]:
from dimod import ConstrainedQuadraticModel
from dimod import cqm_to_bqm
from dimod import Binary

cqm = ConstrainedQuadraticModel()

In [ ]:
K = 10 # K for number of Binary variables for each stock
PK = 1/(2**K)

x = [[Binary(f"x_{i}_{k}") for k in range(K)] for i in range(n)] # Creating Binary variables

In [ ]:
import numpy as np

min_weights = [] # For min weights restriction
max_weights = np.ones(n) # For max weight restriction

for i in range(n):
    min_weights.append( price[i]/B )


risk_term = []
return_term = []

for i in range(n):
    for j in range(n):
        risk_term.append((B*B)*sigma[i][j]*(min_weights[i] + (max_weights[i] - min_weights[i])*(sum(PK * (2**(k - 1)) * x[i][k-1] for k in range(1, K + 1))))* (min_weights[j] + (max_weights[j] - min_weights[j])*(sum(PK * (2**(k - 1)) * x[j][k-1] for k in range(1, K + 1)))))


    return_term.append(mu[i] * (min_weights[i] + (max_weights[i] - min_weights[i])*(sum(PK * (2**(k - 1)) * x[i][k-1] for k in range(1, K + 1))))*B)





In [9]:
cqm.set_objective(q*sum(risk_term) - sum(return_term))

In [10]:
cqm.add_constraint(sum((min_weights[i] + (max_weights[i] - min_weights[i])*(sum(PK * (2**(k - 1)) * x[i][k-1] for k in range(1, K + 1)))) for i in range(n)) == 1, label = "budget_constraint")

'budget_constraint'

In [11]:
bqm, _ = cqm_to_bqm(cqm)

In [12]:
bqm

BinaryQuadraticModel({'x_0_0': -1027441.2775983318, 'x_0_1': -2053668.189394446, 'x_0_2': -4102478.9155800235, 'x_0_3': -8185527.978324571, 'x_0_4': -16293336.54530724, 'x_0_5': -32275795.445246864, 'x_0_6': -63308080.30902327, 'x_0_7': -121642118.2921647, 'x_0_8': -223388067.28080213, 'x_0_9': -367191457.3474951, 'x_1_0': -1027007.2794130828, 'x_1_1': -2052724.7439911193, 'x_1_2': -4100290.2286420544, 'x_1_3': -8179943.419923372, 'x_1_4': -16277338.690403797, 'x_1_5': -32224484.783035804, 'x_1_6': -63128199.17498444, 'x_1_7': -120973316.78562026, 'x_1_8': -220814307.313846, 'x_1_9': -357099309.59811383, 'x_2_0': -1027408.9867215568, 'x_2_1': -2053587.428394601, 'x_2_2': -4102252.6765951533, 'x_2_3': -8184816.632414108, 'x_2_4': -16290878.381723426, 'x_2_5': -32266737.231027704, 'x_2_6': -63273396.33237878, 'x_2_7': -121506480.14605111, 'x_2_8': -222851710.21727633, 'x_2_9': -365058420.1352491, 'x_3_0': -1027779.5985018972, 'x_3_1': -2054474.3931070797, 'x_3_2': -4104609.5706273015, 'x

In [13]:
import neal

sa = neal.SimulatedAnnealingSampler()

sampleset = sa.sample(bqm, num_reads=5000)

In [14]:
sampleset.samples

<bound method SampleSet.samples of SampleSet(rec.array([([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0], 23931893.76145935, 1),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0], 23754356.82123375, 1),
           ([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 24052622.57132149, 1),
           ...,
           ([0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,

In [15]:
result = sampleset.first.sample.values()
result_list = list(result)
result_list

[np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(1),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(1),
 np.int8(1),
 np.int8(0),

In [16]:
resized_result = np.array(result_list).reshape(n,K)
resized_result

array([[0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 1, 0, 1, 0, 0]], dtype=int8)

In [ ]:
## Post processing the result


weights_of_result = []



for i in range(n):
    sum_1 = 0
    
    for k in range(1,K+1):
        sum_1 = sum_1 + (PK * (2**(k-1)) * resized_result[i][k-1])
        
    sum_1 = (sum_1 * (max_weights[i] - min_weights[i])) + min_weights[i]
    weights_of_result.append(sum_1)
    
weights_of_result

[np.float64(0.0822400215011679),
 np.float64(0.045419104564532055),
 np.float64(0.08125393375036696),
 np.float64(0.35648174140345523),
 np.float64(0.12913583826712144),
 np.float64(0.07847828170913766),
 np.float64(6.879791230465767e-05),
 np.float64(0.18268433118723193)]

In [ ]:
print(sum(weights_of_result))   # the budget constraint is not satisfied, but it is near to 1

0.9557620502953177


In [ ]:
#from dimod.binary.BinaryQuadraticModel import to_qubo

qubo = bqm.to_qubo
qubo  # We can use this QUBO to give the problem to Fujitsu DA

<bound method BinaryQuadraticModel.to_qubo of BinaryQuadraticModel({'x_0_0': -1027441.2775983318, 'x_0_1': -2053668.189394446, 'x_0_2': -4102478.9155800235, 'x_0_3': -8185527.978324571, 'x_0_4': -16293336.54530724, 'x_0_5': -32275795.445246864, 'x_0_6': -63308080.30902327, 'x_0_7': -121642118.2921647, 'x_0_8': -223388067.28080213, 'x_0_9': -367191457.3474951, 'x_1_0': -1027007.2794130828, 'x_1_1': -2052724.7439911193, 'x_1_2': -4100290.2286420544, 'x_1_3': -8179943.419923372, 'x_1_4': -16277338.690403797, 'x_1_5': -32224484.783035804, 'x_1_6': -63128199.17498444, 'x_1_7': -120973316.78562026, 'x_1_8': -220814307.313846, 'x_1_9': -357099309.59811383, 'x_2_0': -1027408.9867215568, 'x_2_1': -2053587.428394601, 'x_2_2': -4102252.6765951533, 'x_2_3': -8184816.632414108, 'x_2_4': -16290878.381723426, 'x_2_5': -32266737.231027704, 'x_2_6': -63273396.33237878, 'x_2_7': -121506480.14605111, 'x_2_8': -222851710.21727633, 'x_2_9': -365058420.1352491, 'x_3_0': -1027779.5985018972, 'x_3_1': -205447